In [1]:
import os
import pickle
import numpy as np
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
test = test.fillna(0)

In [3]:
x_train = train[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_train = train.iloc[:,-8:]
x_test = test[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_test = np.asarray(test.iloc[:,-8:])
y_test_decode = []
for i in range(y_test.shape[0]):
    y_test_decode.insert(len(y_test_decode),np.argmax(y_test[i,:]))

In [4]:
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = scaler.fit_transform(x_test)
x_test = np.asarray(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])

In [5]:
# fit model on dataset
def fit_model(trainX, trainy):
    model = Sequential()
    #model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
    model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dense(2000, activation='relu'))
    model.add(Dropout(0.2, noise_shape=None, seed=None))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit([x_train], batch_size=20, y=y_train, verbose=1, validation_split=0.25, shuffle=True, epochs=1)
    return model

In [6]:
# create directory for models
dir_name = "test_models"
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
# fit and save models
n_members = 3
for i in range(n_members):
    # fit model
    model = fit_model(x_train, y_train)
    # save model
    filename = dir_name + '/model_' + str(i + 1) + '.nn'
    model.save(filename)
    print('Saved %s' % filename)

Train on 227768 samples, validate on 75923 samples
Epoch 1/1
227768/227768 [==============================] - 212s 932us/step - loss: 0.9967 - acc: 0.6384 - val_loss: 1.3501 - val_acc: 0.5357
Saved test_models/model_1.nn
Train on 227768 samples, validate on 75923 samples
Epoch 1/1
227768/227768 [==============================] - 208s 915us/step - loss: 1.0040 - acc: 0.6352 - val_loss: 1.3665 - val_acc: 0.5208
Saved test_models/model_2.nn
Train on 227768 samples, validate on 75923 samples
Epoch 1/1
227768/227768 [==============================] - 211s 925us/step - loss: 1.0027 - acc: 0.6342 - val_loss: 1.2763 - val_acc: 0.5688
Saved test_models/model_3.nn
